# Inference

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from arabert.preprocess import ArabertPreprocessor
from transformers import AutoTokenizer
import re

text = "شهدت مدينة طرابلس، مساء أمس الأربعاء، احتجاجات شعبية وأعمال شغب لليوم الثالث على التوالي، وذلك بسبب تردي الوضع المعيشي والاقتصادي. واندلعت مواجهات عنيفة وعمليات كر وفر ما بين الجيش اللبناني والمحتجين استمرت لساعات، إثر محاولة فتح الطرقات المقطوعة، ما أدى إلى إصابة العشرات من الطرفين"

model_name = "aubmindlab/bert-base-arabertv2"
preprocessor = ArabertPreprocessor(model_name=model_name)

def preprocess_function(text):
  return preprocessor.preprocess(text)

def unpreprocess_function(text):
  temp = preprocessor.unpreprocess(text)
  temp = re.sub('\[CLS\]|\[PAD\]|\[SEP\]', '', temp)
  return re.sub('\s+$|^\s', '', temp)


tokenizer = AutoTokenizer.from_pretrained("./AraT5")
model = AutoModelForSeq2SeqLM.from_pretrained("./finetuned-model")


# def tokenize_function(examples):
#   inputs = [preprocess_function(text) for text in examples['text']]
#   inputs = examples['text']
#   model_inputs = tokenizer(inputs, padding='max_length', truncation=True)

#   labels = tokenizer(examples['summary'], padding='max_length', max_length=128, truncation=True, return_tensors="pt")

#   model_inputs['labels'] = labels['input_ids']
#   return model_inputs

inputs = tokenizer(preprocess_function(text), return_tensors="pt").input_ids

outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)
decoded_text=tokenizer.decode(outputs[0], skip_special_tokens=True)
unpreprocessed_text=unpreprocess_function(decoded_text)
print(unpreprocessed_text)
